# 房天下数据爬虫
## 1. import 相关的包

In [ ]:
import requests, bs4, os
import re
import openpyxl

## 2. 使用requests获取数据并保存

In [ ]:
front = r'http://newhouse.cq.fang.com'
after = r'/house/s/b91'
url = front + after

for i in range(38):
    res = requests.get(url)
    res.raise_for_status()
    res.encoding = 'gbk'
    filename = 'page{}'.format(str(i)) + '.txt'
    with open(filename, 'wb') as f:
        for i in res.iter_content(100000):
            f.write(i)`
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    nex = soup.select('.next')
    url = front + nex[0].get('href')

## 3. 使用bs4读取数据，提取相关信息

In [ ]:
wb = openpyxl.load_workbook('data.xlsx')
sheet = wb.get_active_sheet()

for i in range(38):
    f = open('page'+str(i)+'.txt', 'r')
    soup = bs4.BeautifulSoup(f.read(), 'lxml')
    selected = soup.select('.nlc_details')
    data = {}
    n = len(selected)
    for m in range(n):
        try:
            a = selected[m].select('.nlcd_name a')[0].getText()
            a = re.search(re.compile(r'\w+'), a).group()
            data.setdefault('A', a)
        except:
            data.setdefault('A', 'na')
        
        try:
            b = selected[m].select('.address a')[0].getText()
            b = re.search(re.compile(r'\[.+\]'), b).group()
            b = b[1:-1]
            data.setdefault('B', b)
        except:
            data.setdefault('B', 'na')

        try:
            c = selected[m].select('.nhouse_price')[0].getText()
            c = re.search(re.compile(r'\d+'), c).group()
            if len(c) > 3:
                data.setdefault('C', int(c))
            else:
                e = selected[m].select('.house_type')[0].getText()
                e = re.search(re.compile(r'\d+~\d+平米'), e).group()
                # print(e)
                e = re.search(re.compile(r'\d+'), e).group()
                # print(e)
                price = round(int(c + '0000')/int(e))
                # print(price)
                data.setdefault('C', price)
        except:
            data = {}
            continue

        try:
            d = selected[m].select('.value_num')[0].getText()
            d = re.search(re.compile(r'\d+'), d).group()
            data.setdefault('D', int(d))
        except:
            data.setdefault('D', 'na')

        sheet.append(data)
        data = {}

wb.save('data1.xlsx')